In [ ]:
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(file_contents):
    # Inicializar el vectorizador de frecuencia de palabras
    vectorizer = CountVectorizer()

    # Crear la matriz de términos-frecuencia
    tf_matrix = vectorizer.fit_transform(file_contents)

    # Calcular la similitud coseno entre los pares de documentos
    similarity_matrix = cosine_similarity(tf_matrix, tf_matrix)

    return similarity_matrix

def process_files_in_folder(folder_path):
    file_contents = []
    file_paths = []

    # Iterar sobre los archivos en la carpeta
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith('.java'):
                file_path = os.path.join(root, file_name)
                file_paths.append(file_path)
                with open(file_path, 'r') as file:
                    file_contents.append(file.read())

    # Calcular la similitud entre los archivos
    similarity_matrix = calculate_similarity(file_contents)

    # Imprimir la similitud entre los archivos
    print("Similitud entre archivos:")
    for i in range(len(file_paths)):
        for j in range(i+1, len(file_paths)):
            similarity = similarity_matrix[i][j]
            if similarity > 0.9:
                print(f"{file_paths[i]} - {file_paths[j]}: {round(similarity, 2)}")

# Ruta de la carpeta donde se encuentran los archivos .java
folder_path = ".\\conplag\\minor"
process_files_in_folder(folder_path)


## Carga de Archivos con pretratados

In [ ]:
import os
import re

def limpiar_archivo_java(archivo):
    with open(archivo, 'r') as file:
        contenido = file.read()

    # Eliminar sangría y saltos de línea
    contenido = re.sub(r'\n\s*', '\n', contenido)

    # Eliminar comentarios de una línea
    contenido = re.sub(r'//.*', '', contenido)

    # Eliminar comentarios de múltiples líneas
    contenido = re.sub(r'/\*(.|\n)*?\*/', '', contenido)

    # Eliminar líneas de importación de bibliotecas
    contenido = re.sub(r'import\s+.*?;', '', contenido)

     # Eliminar líneas en blanco adicionales
    contenido = re.sub(r'\n\s*\n', '\n', contenido)

    return contenido

carpeta = '.\\conplag\\minor'
archivos_java = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.java')]

for archivo in archivos_java:
    ruta_archivo = os.path.join(carpeta, archivo)
    contenido_limpio = limpiar_archivo_java(ruta_archivo)
    print("Contenido limpio de", archivo, ":\n", contenido_limpio)


### Checkpoint

In [ ]:
import os
import re
import javalang

def limpiar_archivo_java(archivo):
    with open(archivo, 'r') as file:
        contenido = file.read()

    # Eliminar sangría y saltos de línea
    contenido = re.sub(r'\n\s*', '\n', contenido)

    # Eliminar comentarios de una línea
    contenido = re.sub(r'//.*', '', contenido)

    # Eliminar comentarios de múltiples líneas
    contenido = re.sub(r'/\*(.|\n)*?\*/', '', contenido)

    # Eliminar líneas de importación de bibliotecas
    contenido = re.sub(r'import\s+.*?;', '', contenido)

    # Eliminar líneas en blanco adicionales
    contenido = re.sub(r'\n\s*\n', '\n', contenido)

    return contenido

carpeta = '.\\conplag\\minor'
archivos_java = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.java')]

ast_trees = {}

for archivo in archivos_java:
    ruta_archivo = os.path.join(carpeta, archivo)
    contenido_limpio = limpiar_archivo_java(ruta_archivo)
    tokens = javalang.tokenizer.tokenize(contenido_limpio)

    try:
        parser = javalang.parser.Parser(tokens)
        ast_tree = parser.parse_member_declaration()
        ast_trees[archivo] = ast_tree  # Guardar el AST en un diccionario con el nombre del archivo como clave
    except javalang.parser.JavaSyntaxError as e:
        print("Error de sintaxis en", archivo, ":", e)


# Comparación de subestructuras comunes entre los árboles AST almacenados en un diccionario
def comparar_subestructuras_comunes(ast_trees):
    # Diccionario para almacenar las subestructuras comunes encontradas en los árboles
    subestructuras_comunes = {}

    # Recorrer cada par de árboles AST para compararlos
    for nombre_archivo1, arbol1 in ast_trees.items():
        for nombre_archivo2, arbol2 in ast_trees.items():
            # Evitar comparar un árbol consigo mismo
            if nombre_archivo1 != nombre_archivo2:
                # Comparar los árboles AST y encontrar las subestructuras comunes
                subestructuras = encontrar_subestructuras_comunes(arbol1, arbol2)

                # Guardar las subestructuras comunes en el diccionario
                subestructuras_comunes[(nombre_archivo1, nombre_archivo2)] = subestructuras

    return subestructuras_comunes

# Función para encontrar subestructuras comunes entre dos árboles AST
def encontrar_subestructuras_comunes(arbol1, arbol2):
    # Función auxiliar para recorrer los árboles AST recursivamente
    def recorrer_arbol(nodo1, nodo2):
        # Si alguno de los nodos es None, no hay subestructuras comunes aquí
        if nodo1 is None or nodo2 is None:
            return []

        # Si los tipos de los nodos son diferentes, no hay subestructuras comunes aquí
        if type(nodo1) != type(nodo2):
            return []

        # Si los tipos de los nodos son iguales, verifica si son del mismo tipo
        if isinstance(nodo1, javalang.ast.Node) and isinstance(nodo2, javalang.ast.Node):
            # Si los nodos son del mismo tipo, compara sus subnodos recursivamente
            subestructuras_comunes = []
            for subnodo1, subnodo2 in zip(nodo1.children, nodo2.children):
                subestructuras_comunes.extend(recorrer_arbol(subnodo1, subnodo2))
            return subestructuras_comunes

        # Si los nodos son de tipo primitivo (como str, int, etc.), compara sus valores
        if nodo1 == nodo2:
            # Si los valores son iguales, se ha encontrado una subestructura común
            return [(nodo1, nodo2)]
        else:
            # Si los valores son diferentes, no hay subestructuras comunes aquí
            return []

    # Llama a la función auxiliar para iniciar el recorrido recursivo de los árboles
    return recorrer_arbol(arbol1, arbol2)


# Llamada a la función para comparar subestructuras comunes entre los árboles AST
subestructuras_comunes = comparar_subestructuras_comunes(ast_trees)



In [ ]:
def encontrar_subestructuras_comunes(arbol1, arbol2):
    # Función auxiliar para recorrer los árboles AST recursivamente
    def recorrer_arbol(nodo1, nodo2):
        # Si alguno de los nodos es None, no hay subestructuras comunes aquí
        if nodo1 is None or nodo2 is None:
            return []

        # Si los tipos de los nodos son diferentes, no hay subestructuras comunes aquí
        if type(nodo1) != type(nodo2):
            return []

        # Si los tipos de los nodos son iguales, verifica si son del mismo tipo
        if isinstance(nodo1, javalang.ast.Node) and isinstance(nodo2, javalang.ast.Node):
            # Si los nodos son del mismo tipo, compara sus subnodos recursivamente
            subestructuras_comunes = []
            for subnodo1, subnodo2 in zip(nodo1.children, nodo2.children):
                subestructuras_comunes.extend(
                    recorrer_arbol(subnodo1, subnodo2))
            return subestructuras_comunes

        # Si los nodos son de tipo primitivo (como str, int, etc.), compara sus valores
        if nodo1 == nodo2:
            # Si los valores son iguales, se ha encontrado una subestructura común
            return [(nodo1, nodo2)]
        else:
            # Si los valores son diferentes, no hay subestructuras comunes aquí
            return []

    # Llama a la función auxiliar para iniciar el recorrido recursivo de los árboles
    return recorrer_arbol(arbol1, arbol2)


# Comparación de subestructuras comunes entre los árboles AST almacenados en un diccionario
def comparar_subestructuras_comunes(ast_trees):
    # Diccionario para almacenar las subestructuras comunes encontradas en los árboles
    subestructuras_comunes = {}

    # Recorrer cada par de árboles AST para compararlos
    for nombre_archivo1, arbol1 in ast_trees.items():
        for nombre_archivo2, arbol2 in ast_trees.items():
            # Evitar comparar un árbol consigo mismo
            if nombre_archivo1 != nombre_archivo2:
                # Comparar los árboles AST y encontrar las subestructuras comunes
                subestructuras = encontrar_subestructuras_comunes(
                    arbol1, arbol2)

                # Si hay subestructuras comunes, guardar la información
                if subestructuras:
                    # Almacenar el nombre de los archivos equivalentes en un conjunto
                    if nombre_archivo1 not in subestructuras_comunes:
                        subestructuras_comunes[nombre_archivo1] = set()
                    subestructuras_comunes[nombre_archivo1].add(
                        nombre_archivo2)

    return subestructuras_comunes

# Función para imprimir los programas estructuralmente equivalentes


# Función para imprimir los programas estructuralmente equivalentes con puntaje de similitud
def imprimir_programas_equivalentes(subestructuras_comunes):
    print("Programas estructuralmente equivalentes (ordenados por similitud):")
    # Lista para almacenar los programas equivalentes con su puntaje de similitud
    programas_similitud = []
    for nombre_archivo, archivos_equivalentes in subestructuras_comunes.items():
        if archivos_equivalentes:
            # Calcular el puntaje de similitud basado en el número de archivos equivalentes
            similitud = len(archivos_equivalentes)
            programas_similitud.append((nombre_archivo, similitud, archivos_equivalentes))
    
    # Ordenar los programas por similitud (de mayor a menor)
    programas_similitud.sort(key=lambda x: x[1], reverse=True)
    
    # Imprimir los programas estructuralmente equivalentes con su puntaje de similitud
    for programa, similitud, equivalentes in programas_similitud:
        print(f"- {programa} (Similitud: {similitud}):")
        for equivalente in equivalentes:
            print(f"  - {equivalente}")


# Llamada a la función para comparar subestructuras comunes entre los árboles AST
subestructuras_comunes = comparar_subestructuras_comunes(ast_trees)

# Llamada a la función para imprimir los programas estructuralmente equivalentes
imprimir_programas_equivalentes(subestructuras_comunes)

In [ ]:
import os
import re
import javalang
from munkres import Munkres


def limpiar_archivo_java(archivo):
    with open(archivo, 'r') as file:
        contenido = file.read()

    # Eliminar sangría, saltos de línea y comentarios
    contenido = re.sub(r'//.*|/\*(.|\n)*?\*/', '', contenido)

    # Eliminar líneas de importación de bibliotecas
    contenido = re.sub(r'import\s+.*?;', '', contenido)

    # Eliminar líneas en blanco adicionales
    contenido = re.sub(r'\n\s*\n', '\n', contenido)

    return contenido


def get_significant_subtrees(root):
    """Encuentra los subárboles significativos de un AST.

    Args:
    root: La raíz del AST principal.

    Returns:
    Una lista con todos los subárboles significativos de la raíz.
    """
    def is_significant(node):
        """Determina si un nodo AST es significativo."""
        return (
            isinstance(node, javalang.tree.MethodDeclaration) or
            isinstance(node, javalang.tree.IfStatement) or
            isinstance(node, javalang.tree.ClassDeclaration) or
            isinstance(node, javalang.tree.WhileStatement) or
            isinstance(node, javalang.tree.ForStatement) or
            isinstance(node, javalang.tree.ReturnStatement)
        )

    significant_subtrees = []
    for path, node in root.filter(javalang.tree.Node):
        if is_significant(node):
            significant_subtrees.append(node)
    return significant_subtrees


def bestMatchWeight(bestMatch):
    if isinstance(bestMatch, int):  # Si bestMatch es un solo valor entero
        return bestMatch
    else:
        return sum(bestMatch)



def allSubtreeWeight(subtreesa, subtreesb):
    return len(subtreesa) + len(subtreesb)


def kuhnMunkres(costMatrix):
    m = Munkres()
    indexes = m.compute(costMatrix)
    total_cost = sum(costMatrix[i][j] for i, j in indexes)
    return total_cost


def reorderCompare(ASTa, ASTb, depth, visited):
    def get_node_count(node, visited):
        """Función auxiliar para contar el número de nodos."""
        stack = [(node, ())]
        count = 0
        while stack:
            current_node, path = stack.pop()
            # Usar la tupla (path, type(current_node)) como identificador único del nodo
            node_id = (path, type(current_node))
            if node_id in visited:
                continue
            visited.add(node_id)
            count += 1
            if isinstance(current_node, javalang.tree.Node):
                # Actualizar el path con la ubicación del nodo actual
                new_path = path + (current_node,)
                stack.extend((child, new_path) for child in current_node.children)
        return count



    node_count_a = get_node_count(ASTa, visited)
    node_count_b = get_node_count(ASTb, visited)

    if node_count_a <= 1 or node_count_b <= 1:
        return compareASTs(ASTa, ASTb, depth, visited) + 1
    else:
        costMatrix = [[compareASTs(child_a, child_b, depth, visited)
                       for child_b in ASTb] for child_a in ASTa]
        bestMatch = kuhnMunkres(costMatrix)
        return sum(bestMatch)


def compareASTs(ASTa, ASTb, depth, visited):
    def get_node_count(node, visited):
        """Función auxiliar para contar el número de nodos."""
        stack = [(node, ())]
        count = 0
        while stack:
            current_node, path = stack.pop()
            # Usar la tupla (path, type(current_node)) como identificador único del nodo
            node_id = (path, type(current_node))
            if node_id in visited:
                continue
            visited.add(node_id)
            count += 1
            if isinstance(current_node, javalang.tree.Node):
                # Actualizar el path con la ubicación del nodo actual
                new_path = path + (current_node,)
                stack.extend((child, new_path) for child in current_node.children)
        return count

    node_count_a = get_node_count(ASTa, visited)
    node_count_b = get_node_count(ASTb, visited)

    if isinstance(ASTa, type(ASTb)) and node_count_a == node_count_b == 0:
        return 1
    if not isinstance(ASTa, type(ASTb)) or node_count_a != node_count_b:
        return 0
    if depth == 0:
        return sum(compareASTs(child_a, child_b, depth, visited) for child_a, child_b in zip(ASTa, ASTb)) + 1
    elif depth > 0:
        return reorderCompare(ASTa, ASTb, depth - 1, visited) + 1


def compareSubtrees(subtreesa, subtreesb, depth):
    visited = set()
    node_count_a = len(subtreesa)
    node_count_b = len(subtreesb)

    if node_count_a <= 1 or node_count_b <= 1:
        bestMatch = compareASTs(subtreesa, subtreesb, depth, visited)
    else:
        costMatrix = [[compareASTs(child_a, child_b, depth, visited)
                       for child_b in subtreesb] for child_a in subtreesa]
        bestMatch = kuhnMunkres(costMatrix)

    return 2 * bestMatchWeight(bestMatch) / allSubtreeWeight(subtreesa, subtreesb)


# Comparar los árboles de sintaxis abstracta
carpeta = '.\\conplag\\minor'
archivos_java = [archivo for archivo in os.listdir(
    carpeta) if archivo.endswith('.java')]

ast_trees = {}

for archivo in archivos_java:
    ruta_archivo = os.path.join(carpeta, archivo)
    contenido_limpio = limpiar_archivo_java(ruta_archivo)
    tokens = javalang.tokenizer.tokenize(contenido_limpio)

    try:
        parser = javalang.parser.Parser(tokens)
        ast_tree = parser.parse_member_declaration()
        # Obtener los nodos significativos del árbol AST
        significant_subtrees = get_significant_subtrees(ast_tree)
        # Guardar los nodos significativos en un diccionario con el nombre del archivo como clave
        ast_trees[archivo] = significant_subtrees
    except javalang.parser.JavaSyntaxError as e:
        print("Error de sintaxis en", archivo, ":", e)

for archivo_a, subtrees_a in ast_trees.items():
    for archivo_b, subtrees_b in ast_trees.items():
        if archivo_a != archivo_b:
            similarity = compareSubtrees(subtrees_a, subtrees_b, depth=3)
            print(f"Similitud entre {archivo_a} y {archivo_b}: {similarity}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import javalang
import numpy as np
from scipy.spatial.distance import cosine


def limpiar_archivo_java(archivo):
    with open(archivo, 'r') as file:
        contenido = file.read()

    # Eliminar sangría y saltos de línea
    contenido = re.sub(r'\n\s*', '\n', contenido)

    # Eliminar comentarios de una línea
    contenido = re.sub(r'//.*', '', contenido)

    # Eliminar comentarios de múltiples líneas
    contenido = re.sub(r'/\*(.|\n)*?\*/', '', contenido)

    # Eliminar líneas de importación de bibliotecas
    contenido = re.sub(r'import\s+.*?;', '', contenido)

    # Eliminar líneas en blanco adicionales
    contenido = re.sub(r'\n\s*\n', '\n', contenido)

    return contenido


def ted(tree1, tree2):
    n = len(tree1)
    m = len(tree2)

    # Inicializar la matriz de distancia
    dp = [[0] * (m + 1) for _ in range(n + 1)]

    # Llenar la primera fila y la primera columna
    for i in range(n + 1):
        dp[i][0] = i
    for j in range(m + 1):
        dp[0][j] = j

    # Calcular la distancia de edición
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost = 0 if tree1[i - 1] == tree2[j - 1] else 1
            dp[i][j] = min(dp[i - 1][j] + 1, dp[i][j - 1] +
                           1, dp[i - 1][j - 1] + cost)

    # Reconstruir las operaciones de edición
    operations = []
    i, j = n, m
    while i > 0 or j > 0:
        if i > 0 and dp[i][j] == dp[i - 1][j] + 1:
            operations.append(f'Delete node {i}')
            i -= 1
        elif j > 0 and dp[i][j] == dp[i][j - 1] + 1:
            operations.append(f'Insert node {j}')
            j -= 1
        else:
            if tree1[i - 1] != tree2[j - 1]:
                operations.append(f'Replace node {i} with {tree2[j - 1]}')
            i -= 1
            j -= 1

    # Invertir las operaciones ya que fueron construidas desde el final
    operations = operations[::-1]

    return dp[n][m], operations


carpeta = '.\\\\conplag\\\\minor'
archivos_java = [archivo for archivo in os.listdir(
    carpeta) if archivo.endswith('.java')]

ast_trees = {}

# Función para convertir el AST de un archivo Java en una lista de nodos


def ast_to_list(ast):
    if isinstance(ast, javalang.ast.Node):
        return [ast_to_list(child) for child in ast.children]
    else:
        return ast


# Generar árboles AST para todos los archivos Java en la carpeta
for archivo in archivos_java:
    ruta_archivo = os.path.join(carpeta, archivo)
    contenido_limpio = limpiar_archivo_java(ruta_archivo)
    tokens = javalang.tokenizer.tokenize(contenido_limpio)

    try:
        parser = javalang.parser.Parser(tokens)
        ast_tree = parser.parse_member_declaration()
        # Convertir el AST en una lista de nodos y guardar en un diccionario con el nombre del archivo como clave
        ast_trees[archivo] = ast_to_list(ast_tree)
    except javalang.parser.JavaSyntaxError as e:
        print("Error de sintaxis en", archivo, ":", e)

dist_matrix = np.zeros((len(ast_trees), len(ast_trees)))
for i, (archivo1, tree1) in enumerate(ast_trees.items()):
    for j, (archivo2, tree2) in enumerate(ast_trees.items()):
        if i != j:
            distance, _ = ted(tree1, tree2)
            dist_matrix[i, j] = distance

# Convertir la matriz de distancia en una matriz de similitud
sim_matrix = 1 / (1 + dist_matrix)

# Calcular la similitud de cosenos entre los archivos
similarities = np.zeros((len(ast_trees), len(ast_trees)))
for i in range(len(ast_trees)):
    for j in range(len(ast_trees)):
        if i != j:
            similarities[i, j] = 1 - cosine(sim_matrix[i], sim_matrix[j])


# Crear un mapa de calor de la matriz de similitud
# Obtener los nombres de los archivos como una lista ordenada
archivos = list(ast_trees.keys())

# Crear un mapa de calor de la matriz de similitud
sns.set(style="white")
plt.figure(figsize=(10, 8))
ax = sns.heatmap(similarities, annot=True,
                 cmap="YlGnBu", fmt=".2f", linewidths=.5)
ax.set_xticklabels(archivos, rotation=45, ha='right')
ax.set_yticklabels(archivos, rotation=0)
plt.title("Matriz de similitud entre archivos")
plt.xlabel("Archivos")
plt.ylabel("Archivos")
plt.show()

# Conjunto para almacenar los nombres de los archivos procesados
archivos_procesados = set()

# Comparar todos los pares de archivos
for archivo1 in ast_trees:
    # Agregar el archivo1 al conjunto de archivos procesados
    archivos_procesados.add(archivo1)
    for archivo2 in ast_trees:
        # Verificar que archivo2 no esté en el conjunto de archivos procesados
        if archivo1 != archivo2 and archivo2 not in archivos_procesados:
            distance, edit_operations = ted(
                ast_trees[archivo1], ast_trees[archivo2])
            print(f"Distancia entre {archivo1} y {archivo2}: {distance}")

## Similitud Cosenos (Matriz de Similitud)

In [23]:
import os
import javalang
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def limpiar_archivo_java(archivo):
    with open(archivo, 'r') as file:
        contenido = file.read()

    # Eliminar sangría y saltos de línea
    contenido = re.sub(r'\n\s*', '\n', contenido)

    # Eliminar comentarios de una línea
    contenido = re.sub(r'//.*', '', contenido)

    # Eliminar comentarios de múltiples líneas
    contenido = re.sub(r'/\*(.|\n)*?\*/', '', contenido)

    # Eliminar líneas de importación de bibliotecas
    contenido = re.sub(r'import\s+.*?;', '', contenido)

    # Eliminar líneas en blanco adicionales
    contenido = re.sub(r'\n\s*\n', '\n', contenido)

    return contenido

def generate_file_contents(folder_path):
    file_contents = {}
    files = [file for file in os.listdir(folder_path) if file.endswith('.java')]

    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        cleaned_content = limpiar_archivo_java(file_path)
        file_contents[file_name] = cleaned_content

    return file_contents

def calculate_cosine_similarity(file_contents):
    # Convertir los contenidos de los archivos a una lista
    corpus = list(file_contents.values())

    # Utilizar TfidfVectorizer para obtener las características de los archivos
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # Calcular la matriz de similitud de cosenos entre los archivos
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

    return similarity_matrix

# Directorio que contiene los archivos Java
folder_path = ".\\CFiles\\java\\train"

# Obtener los contenidos de los archivos
file_contents = generate_file_contents(folder_path)

# Calcular la similitud de cosenos entre los archivos
similarity_matrix = calculate_cosine_similarity(file_contents)

# Obtener los nombres de los archivos
file_names = list(file_contents.keys())

# Iterar sobre la matriz de similitud de cosenos y encontrar pares de archivos similares
for i in range(len(file_names)):
    for j in range(i + 1, len(file_names)):
        if similarity_matrix[i, j] > 0.85:
            print(f"Archivos similares: {file_names[i]} y {file_names[j]}, Similaridad: {similarity_matrix[i, j]}")

# plt.figure(figsize=(12, 10))
# heatmap = sns.heatmap(similarity_matrix, cmap="YlGnBu", annot=True, fmt=".2f", linewidths=.5)
# plt.title("Matriz de similitud de cosenos entre archivos")
# plt.xlabel("Archivos")
# plt.ylabel("Archivos")
# plt.show()

Archivos similares: 005.java y 006.java, Similaridad: 0.999752241273496
Archivos similares: 014.java y 021.java, Similaridad: 0.9991661158784219
Archivos similares: 015.java y 023.java, Similaridad: 1.0000000000000002
Archivos similares: 016.java y 024.java, Similaridad: 0.9960228996159711
Archivos similares: 017.java y 022.java, Similaridad: 0.9783653993364245
Archivos similares: 030.java y 032.java, Similaridad: 0.8866578225067605
Archivos similares: 043.java y 251.java, Similaridad: 0.9891486882761814
Archivos similares: 048.java y 051.java, Similaridad: 0.9699607980870207
Archivos similares: 048.java y 059.java, Similaridad: 0.8663840165899407
Archivos similares: 051.java y 059.java, Similaridad: 0.8825356416023545
Archivos similares: 059.java y 183.java, Similaridad: 0.8773749974089675
Archivos similares: 059.java y 258.java, Similaridad: 0.9124964895215663
Archivos similares: 062.java y 064.java, Similaridad: 0.8986972219253097
Archivos similares: 101.java y 212.java, Similaridad

## Similitud Cosenos (ASTs)

In [24]:
import os
import re
import javalang

def limpiar_archivo_java(archivo):
    with open(archivo, 'r') as file:
        contenido = file.read()

    # Eliminar comentarios de una línea
    contenido = re.sub(r'//.*', '', contenido)

    # Eliminar comentarios de múltiples líneas
    contenido = re.sub(r'/\*(.|\n)*?\*/', '', contenido)

    # Eliminar líneas de importación de bibliotecas
    contenido = re.sub(r'import\s+.*?;', '', contenido)

    # Eliminar espacios dentro de cadenas de texto
    contenido = re.sub(r'"([^"\\]*(?:\\.[^"\\]*)*)"', lambda m: m.group(0).replace(' ', '\\'), contenido)

    # Escapar espacios en blanco dentro de cadenas de caracteres
    contenido = re.sub(r'(\".*?)(\s+)(.*?\")', lambda match: match.group(1) + match.group(2).replace(' ', '\\ ') + match.group(3), contenido)

    # Eliminar líneas en blanco adicionales
    contenido = re.sub(r'\n\s*\n', '\n', contenido)

    return contenido

def cargar_archivos_java(folder_path):
    file_contents = {}
    archivos_java = [archivo for archivo in os.listdir(folder_path) if archivo.endswith('.java')]

    for archivo in archivos_java:
        ruta_archivo = os.path.join(folder_path, archivo)
        contenido_limpio = limpiar_archivo_java(ruta_archivo)
        file_contents[archivo] = contenido_limpio

    return file_contents

# Directorio que contiene los archivos Java
folder_path = ".\\conplag\\minor"
file_contents = cargar_archivos_java(folder_path)

# Mostrar el contenido de los archivos cargados y construir el AST
for nombre_archivo, contenido_archivo in file_contents.items():
    print(f"Archivo: {nombre_archivo}")
    print(contenido_archivo)
    print("------------------------------------")
    
    # Construir el AST utilizando javalang
    try:
        ast = javalang.parse.parse(contenido_archivo)
        print("AST:")
        print(ast)
    except javalang.parser.JavaSyntaxError as e:
        print("Error de sintaxis en el archivo:", e)
    print("------------------------------------")
    


Archivo: 115c99cb.java

public class D_Java {
	public static final int MOD = 998244353;
	public static int mul(int a, int b) {
		return (int)((long)a * (long)b % MOD);
	}
	int[] f;
	int[] rf;
	public int C(int n, int k) {
		return (k < 0 || k > n) ? 0 : mul(f[n], mul(rf[n-k], rf[k]));
	}
	public static int pow(int a, int n) {
		int res = 1;
		while (n != 0) {
			if ((n & 1) == 1) {
				res = mul(res, a);
			}
			a = mul(a, a);
			n >>= 1;
		}
		return res;
	}
	public static int inv(int a) {
		return pow(a, MOD-2);
	}
	public void doIt() throws IOException {
		BufferedReader in = new BufferedReader(new InputStreamReader(System.in));
		StringTokenizer tok = new StringTokenizer(in.readLine());
		int n = Integer.parseInt(tok.nextToken());
		int k = Integer.parseInt(tok.nextToken());
		f = new int[n+42];
		rf = new int[n+42];
		f[0] = rf[0] = 1;
		for (int i = 1; i < f.length; ++i) {
			f[i] = mul(f[i-1], i);
			rf[i] = mul(rf[i-1], inv(i));
		}
		int[] events = new int[2*n];
		for (int i =

In [29]:
from scipy.spatial.distance import cosine
import numpy as np
import os
import re
import javalang


def limpiar_archivo_java(archivo):
    with open(archivo, 'r') as file:
        contenido = file.read()

    # Eliminar sangría y saltos de línea
    contenido = re.sub(r'\n\s*', '\n', contenido)

    # Eliminar comentarios de una línea
    contenido = re.sub(r'//.*', '', contenido)

    # Eliminar comentarios de múltiples líneas
    contenido = re.sub(r'/\*(.|\n)*?\*/', '', contenido)

    # Eliminar líneas de importación de bibliotecas
    contenido = re.sub(r'import\s+.*?;', '', contenido)

    # Eliminar líneas en blanco adicionales
    contenido = re.sub(r'\n\s*\n', '\n', contenido)

    return contenido


carpeta = ".\\conplag\\minor"
archivos_java = [archivo for archivo in os.listdir(
    carpeta) if archivo.endswith('.java')]

ast_trees = {}

for archivo in archivos_java:
    ruta_archivo = os.path.join(carpeta, archivo)
    contenido_limpio = limpiar_archivo_java(ruta_archivo)
    tokens = javalang.tokenizer.tokenize(contenido_limpio)

    try:
        parser = javalang.parser.Parser(tokens)
        ast_tree = parser.parse_member_declaration()
        # Guardar el AST en un diccionario con el nombre del archivo como clave
        ast_trees[archivo] = ast_tree
    except javalang.parser.JavaSyntaxError as e:
        print("Error de sintaxis en", archivo, ":", e)

## Conteo de tipos de datos totales

In [30]:
def contar_tipos_de_datos(ast_trees):
    tipos_de_datos = {}

    for archivo, ast_tree in ast_trees.items():
        # Recorrer el árbol sintáctico abstracto
        for _, node in ast_tree:
            # Obtener el tipo de nodo
            tipo = type(node).__name__
            # Incrementar el contador para este tipo de dato
            tipos_de_datos[tipo] = tipos_de_datos.get(tipo, 0) + 1

    return tipos_de_datos

# Obtener el conteo de tipos de datos
conteo_tipos_de_datos = contar_tipos_de_datos(ast_trees)

# Mostrar los resultados
for tipo, conteo in conteo_tipos_de_datos.items():
    print(f"{tipo}: {conteo}")

ClassDeclaration: 9
FieldDeclaration: 15
BasicType: 193
VariableDeclarator: 140
Literal: 204
MethodDeclaration: 30
FormalParameter: 41
ReturnStatement: 17
Cast: 13
BinaryOperation: 212
MemberReference: 571
TernaryExpression: 6
MethodInvocation: 153
ArraySelector: 94
LocalVariableDeclaration: 93
WhileStatement: 10
BlockStatement: 65
IfStatement: 36
StatementExpression: 133
Assignment: 95
ReferenceType: 64
ClassCreator: 30
ArrayCreator: 16
ForStatement: 27
ForControl: 27
VariableDeclaration: 27
TypeArgument: 4
BreakStatement: 4
ConstructorDeclaration: 1
This: 2


In [31]:
def contar_tipos_de_datos_por_archivo(ast_trees):
    conteo_por_archivo = {}

    for archivo, ast_tree in ast_trees.items():
        conteo_tipos_de_datos = {}
        # Recorrer el árbol sintáctico abstracto
        for _, node in ast_tree:
            # Obtener el tipo de nodo
            tipo = type(node).__name__
            # Incrementar el contador para este tipo de dato
            conteo_tipos_de_datos[tipo] = conteo_tipos_de_datos.get(tipo, 0) + 1

        # Guardar el conteo para este archivo
        conteo_por_archivo[archivo] = conteo_tipos_de_datos

    return conteo_por_archivo

# Obtener el conteo de tipos de datos por archivo
conteo_tipos_de_datos_por_archivo = contar_tipos_de_datos_por_archivo(ast_trees)

# Mostrar los resultados
for archivo, conteo_tipos in conteo_tipos_de_datos_por_archivo.items():
    print(f"Archivo: {archivo}")
    for tipo, conteo in conteo_tipos.items():
        print(f"\t{tipo}: {conteo}")


Archivo: 115c99cb.java
	ClassDeclaration: 1
	FieldDeclaration: 3
	BasicType: 33
	VariableDeclarator: 18
	Literal: 29
	MethodDeclaration: 6
	FormalParameter: 8
	ReturnStatement: 4
	Cast: 3
	BinaryOperation: 34
	MemberReference: 93
	TernaryExpression: 1
	MethodInvocation: 24
	ArraySelector: 14
	LocalVariableDeclaration: 12
	WhileStatement: 2
	BlockStatement: 8
	IfStatement: 4
	StatementExpression: 22
	Assignment: 18
	ReferenceType: 8
	ClassCreator: 5
	ArrayCreator: 3
	ForStatement: 3
	ForControl: 3
	VariableDeclaration: 3
Archivo: 2bbf754b.java
	ClassDeclaration: 2
	MethodDeclaration: 1
	FormalParameter: 3
	ReferenceType: 10
	LocalVariableDeclaration: 11
	VariableDeclarator: 16
	ClassCreator: 3
	MemberReference: 35
	BasicType: 16
	MethodInvocation: 11
	WhileStatement: 1
	BinaryOperation: 7
	Literal: 11
	BlockStatement: 8
	ArrayCreator: 1
	ForStatement: 3
	ForControl: 3
	VariableDeclaration: 3
	StatementExpression: 9
	Assignment: 5
	ArraySelector: 2
	TypeArgument: 2
	IfStatement: 3
	Break